In [44]:
import pandas as pd
df = pd.read_excel("cleaned_Dogfood.xlsx")

In [45]:
# all attributes to get facters
def get_catergories(data, category): # input: "str"
        # return a list of catergories
        return list(set(data[category]))
    
def get_sns_rank(data, col): # input: data = DataFrame, col = list["str"..]
        # return a list of the order of sns avg group by catergories increasing order
        rank = data.groupby(col).sns.mean().sort_values().index
        return list(rank)

def get_market_percentage(data, col):# input: data = DataFrame, col = list["str"..]
        # return the market values sum of each catergories
        return data.groupby(col).total_price.sum()

def get_proportions(data, col):# input: data = DataFrame, col = list["str"..]
        # return the proportions of given input (1 or 2)
        if len(col) == 1:
                return data.groupby(col).size() / len(data)
        # for 2 inputs, the order decide which is the main(1st) and sub(2nd) category
        return data.groupby(col).size().groupby(level=0, group_keys=False).apply(lambda x: x/x.sum())

In [79]:
get_market_percentage(df, "gender")

gender
F    185830.29
M    178319.49
Name: total_price, dtype: float64

In [46]:
# given age and return the correspond age group
def age_categorize(age):
    if age in range(18, 25):
        age_group = '18-24'
    elif age in range(25, 35):
        age_group = '25-34'
    elif age in range(35, 45):
        age_group = '35-44'
    elif age in range(45, 55):
        age_group = '45-54'
    elif age in range(55, 65):
        age_group = '55-64'
    elif age >= 65:
        age_group = '65+'
    elif age < 18:
        raise ValueError("Too young to get WOOF!")
    return age_group

# given income and return income group
def income_categorize(income):    
    if income < 20000:
        income_group = 'Less than $20,000'
    elif income in range(20000, 40000):
        income_group = '$20,000 - $39,999'
    elif income in range(40000, 60000):
        income_group = '$40,000 - $59,999'
    elif income in range(60000, 80000):
        income_group = '$60,000 - $79,999'
    elif income in range(80000, 100000):
        income_group = '$80,000 - $99,999'
    elif income >= 100000:
        income_group = '$100,000 or more'
    return income_group

#given price and return price group
def price_categorize(price):
    if price > 0 and price <= 19.99:
        price_group = 'cheap'
    elif price > 19.99 and price <= 39.17:
        price_group = 'middle'
    elif price > 39.17 and price <= 56.04:
        price_group = 'good'
    elif price > 56.04 and price <= 110:
        price_group = 'premium'
    elif price > 100:
        raise ValueError('Price too expensive!')
    return price_group

In [47]:
#demo demo-geographic helper function
def dg_validate(demogeo):
    # get input info of gender, age, state
    # output the column name needed
    column = []
    for gas in demogeo:
        if gas in {'M', 'F'}:
            column.append("gender")
            continue
        elif gas in {'55-64', '35-44', '25-34', '18-24', '65+', '45-54'}:
            column.append("age")
            continue
        elif gas in {'UT', 'AL', 'OR', 'MO', 'MT', 'WY', 'MN', 'SD', 'PA', 'CO', 'OK',
                   'HI', 'WA', 'SC', 'IL', 'WV', 'MI', 'WI', 'CA', 'MS', 'KS', 'ID',
                   'OH', 'IN', 'NM', 'VA', 'GA', 'LA', 'DC', 'ND', 'NY', 'MD', 'AK',
                   'DE', 'NV', 'AZ', 'IA', 'NC', 'NE', 'AR', 'FL', 'TX', 'KY', 'TN'}:
            column.append("st")
            continue
        else: # input validation
            raise ValueError("Invalid Args Input")
    return column

In [72]:
# behavior factors consideration
def behavior_validation(data, age = None, ad_type = None):
    # only validate the affect of advertisement option
    if not age and ad_type:
        return (get_sns_rank(data, "ad_exp").index(ad_type) + 1) / 6
    # only validate the affect of age group
    if not ad_type and age:
        return (get_sns_rank(data, "age").index(age_categorize(age)) + 1) / 6
    # calculate the sns affect by age group and ad type
    filter_age = data[data['age'] == age_categorize(age)]
    ad_count = get_sns_rank(filter_age, "ad_exp")
    behavior_score = (ad_count.index(ad_type) + 1) / 6
    return behavior_score

In [49]:
#demo pyschographic helper function
def pyscho_validate(pyscho):
    column = []
    # get input info of income target and price
    # output the column name needed
    for pi in pyscho: 
        if pi in {'$20,000 - $39,999', '$40,000 - $59,999', '$60,000 - $79,999',
                  '$80,000 - $99,999', 'Less than $20,000', '$100,000 or more'}:
            column.append("income")
        elif pi in {'middle', 'cheap', 'good', 'premium'}:
            column.append("price_level")
        else: # input validation
            raise ValueError("Invalid Args Input")
    return column

In [67]:
def woof(data, brand, demo_geo, behavioral, pyscho):
        DG, B, P = 0.3, 0.4, 0.3
        # calculate the competebility of brand within all brands
        woof_score = 0
        brand_score = get_sns_rank(data, "product_brand").index(brand) / len(get_catergories(data, "product_brand"))
        brand_market = get_market_percentage(data, "product_brand")[brand]
        # calculate the demo-geographic impact score
        if demo_geo:
                DG = 0
                demo_geo_score = get_market_percentage(data, dg_validate(demo_geo))
                for dg in demo_geo:
                        demo_geo_score = demo_geo_score.loc[dg]
                demo_geo_score = demo_geo_score/brand_market
                woof_score += (demo_geo_score * 0.3)
                print("dg = {}".format(demo_geo_score))
        # calculate the behavioral impact score
        if behavioral:
                B = 0
                if len(behavioral) < 2:
                        behavioral_score = behavior_validation(data, behavioral[0]) * 0.4
                else:
                        behavioral_score = behavior_validation(data, behavioral[0], behavioral[1]) * 0.4
                woof_score += behavioral_score
                print("behav = {}".format(behavioral_score))
        # calculate the pyschographic impact score
        if pyscho:
                P = 0
                pyscho_score = get_proportions(data, pyscho_validate(pyscho))
                for p in pyscho:
                        pyscho_score = pyscho_score.loc[p]
                woof_score += pyscho_score
                print("pys = {}".format(pyscho_score))
        woof_score += (brand_score * (DG + B + P))
        print(brand_score)
        print("Your WOOF score is {}!!".format(woof_score))

In [52]:
# fit all input of WOOF function into our woof score model
def WOOF(data, brand, gender = None, age = None, state = None, ad_type = None, price = None, income = None):
    if age: age_group = age_categorize(age)
    if income: income_group = income_categorize(income)
    if price: price_group = price_categorize(price)
    if gender or age or state:
        demo_geo = []
        if gender: demo_geo.append(gender)
        if age: demo_geo.append(age_group)
        if state: demo_geo.append(state)
    else: demo_geo = None
    if age or ad_type:
        behav = []
        if age: behav.append(age)
        if ad_type: behav.append(ad_type)
    else: behav = None
    if price or income:
        pyscho = []
        if price: pyscho.append(price_group)
        if income: pyscho.append(income_group)
    else: pyscho = None
    return woof(data, brand, demo_geo, behav, pyscho)

In [80]:
WOOF(df, "King", gender="F")

dg = 16.849165612632092
0.76
Your WOOF score is 5.586749683789628!!


In [ ]:
# put products to correspond brands in a dictionary
df = pd.read_excel("cleaned_Dogfood.xlsx")
products = set(df.product_name)
brands = set(df.product_brand)
brand_products = {}
for brand in brands:
    brand_products[brand] = []
for product in products:
    clip = product.split()[0:2]
    clips = " ".join(clip)
    for brand in brands:
        if brand in clips:
            brand_products[brand].append(product)
            break
brand_products